# Keyword Trends from ATC data

In [1]:
import time
import datetime
from datetime import datetime
import pandas as pd
import numpy as np
from datetime import date, timezone, timedelta

import collections
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

import pytz
import json
import logging
from pathlib2 import Path
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

working_folder=str(Path.cwd())
print(str(working_folder))

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

c:\Users\Mattheus\github\atc-trend


## Import ATC data

In [11]:
day_d0_df = pd.read_csv('atc_data.csv', index_col=0)

## Identify Keywords

In [12]:
startTime = time.time()

### CLEAN QUERIED D-0 ATC RAW DATA ###
day_d0_clean_left_1 = day_d0_df
to_remove = ['ready','stock','in malaysia']
pat = r'({})'.format('|'.join(to_remove))
day_d0_clean_left_1['name'] = day_d0_clean_left_1['name'].str.lower().str.replace(pat, '', regex=True).fillna(day_d0_clean_left_1['name'])

### SUM ATC COLUMN ###
# day_d0_clean_left_1 = day_d0_clean_left_1[day_d0_clean_left_1['level1_category'] == 'Home Appliances']
day_d0_clean_left_2 = day_d0_clean_left_1.reindex(day_d0_clean_left_1.index.repeat(day_d0_clean_left_1.atc))

# specify n-grams
c_vec = CountVectorizer(ngram_range=(2, 3))
# input to fit_transform() should be an iterable with strings
ngrams = c_vec.fit_transform(day_d0_clean_left_2['name'].to_list())
# needs to happen after fit_transform()
vocab = c_vec.vocabulary_
terms = c_vec.get_feature_names()
freqs = ngrams.sum(axis=0).A1
result = dict(zip(terms, freqs))

day_d0_final_left = pd.DataFrame(result.items(), columns=['keywords', 'count']).sort_values(by=['count'], ascending = False)
day_d0_final_left['totalwords'] = day_d0_final_left['keywords'].str.split().str.len()
day_d0_final_left['max_totalwords'] = day_d0_final_left.groupby('count')['totalwords'].transform(max)
day_d0_final_left = day_d0_final_left[day_d0_final_left['totalwords'] == day_d0_final_left['max_totalwords']]
day_d0_final_left = day_d0_final_left.rename({'count': 'atc',}, axis=1)
day_d0_final_left = day_d0_final_left[day_d0_final_left.columns.drop(list(day_d0_final_left.filter(regex='totalwords')))]
day_d0_final_left['ranking'] = np.arange(len(day_d0_final_left)) + 1

### SUM PURCHASED COLUMN ###
day_d0_clean_right = day_d0_clean_left_1[day_d0_clean_left_1['purchased'] > 0]
day_d0_clean_right = day_d0_clean_right.reindex(day_d0_clean_right.index.repeat(day_d0_clean_right.purchased))

# specify n-grams
c_vec = CountVectorizer(ngram_range=(2, 3))
# input to fit_transform() should be an iterable with strings
ngrams = c_vec.fit_transform(day_d0_clean_right['name'].to_list())
# needs to happen after fit_transform()
vocab = c_vec.vocabulary_
terms = c_vec.get_feature_names()
freqs = ngrams.sum(axis=0).A1
result = dict(zip(terms, freqs))

day_d0_final_right = pd.DataFrame(result.items(), columns=['keywords', 'count']).sort_values(by=['count'], ascending = False)
day_d0_final_right['totalwords'] = day_d0_final_right['keywords'].str.split().str.len()
day_d0_final_right['max_totalwords'] = day_d0_final_right.groupby('count')['totalwords'].transform(max)
day_d0_final_right = day_d0_final_right[day_d0_final_right['totalwords'] == day_d0_final_right['max_totalwords']]
day_d0_final_right = day_d0_final_right.rename({'count': 'purchased',}, axis=1)
day_d0_final_right = day_d0_final_right[day_d0_final_right.columns.drop(list(day_d0_final_right.filter(regex='totalwords')))]

### COMBINE ATC & PURCHASED ###
day_d0_final = day_d0_final_left.set_index('keywords').join(day_d0_final_right.set_index('keywords'), how='left')
day_d0_final.head(5)

endTime = time.time()
totalTime = endTime - startTime
logger.info(f'Time Elapsed: {time.strftime("%H:%M:%S", time.gmtime(totalTime))}')

INFO:__main__:Time Elapsed: 00:04:25


## ATC Keyword Trend

In [16]:
day_d0_final.head(10)

,atc,ranking,purchased
keywords,,,
plus size,113168,1,38231.0
long sleeve,110893,2,33285.0
pro max,108560,3,43337.0
stainless steel,96875,4,36561.0
high quality,83212,5,31588.0
xs max,80786,6,31565.0
phone case,76557,7,32497.0
short sleeve,74627,8,26554.0
11 pro,72443,9,28490.0
